In [ ]:
!pip install spuco

Henry Liu - George Implementation With SpuCo

In [ ]:
import torch
from torch.optim import SGD
from spuco.datasets import SpuCoMNIST, SpuriousFeatureDifficulty
from spuco.models import model_factory
from spuco.datasets.group_labeled_dataset_wrapper import GroupLabeledDatasetWrapper
from spuco.group_inference.george_utils.cluster import OverclusterModel
from spuco.robust_train.group_dro import GroupDRO
from spuco.group_inference.cluster import ClusterAlg
from spuco.evaluate import Evaluator
from spuco.utils import set_seed,convert_labels_to_partition
from spuco.robust_train import ERM
import umap
from sklearn.preprocessing import StandardScaler

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
set_seed(0)

In [ ]:
classes = [[0, 1], [2, 3], [4, 5], [6, 7], [8, 9]]
difficulty = SpuriousFeatureDifficulty.MAGNITUDE_LARGE

trainset = SpuCoMNIST(
    root="./content/data/mnist/",
    spurious_feature_difficulty=difficulty,
    spurious_correlation_strength=0.995,
    classes=classes,
    split="train"
)
trainset.initialize()

valset = SpuCoMNIST(
    root="./content/data/mnist/",
    spurious_feature_difficulty=difficulty,
    classes=classes,
    split="val",
)
valset.initialize()
testset = SpuCoMNIST(
    root="./content/data/mnist/",
    spurious_feature_difficulty=difficulty,
    classes=classes,
    split="test"
)
testset.initialize()

In [ ]:
model = model_factory("lenet", trainset[0][0].shape, trainset.num_classes).to(device)
ermtrainer = ERM(
    model=model,
    trainset = trainset,
    batch_size = 128,
    optimizer = SGD(model.parameters(),
                    lr = 0.01,
                    momentum=0.9,
                    nesterov=True),
    num_epochs = 20,
    device=device,
    verbose=True
)
ermtrainer.train()

Epoch 0:   6%|▌         | 23/376 [00:00<00:10, 34.38batch/s, accuracy=87.5%, loss=1.28] 

ERM | Epoch 0 | Loss: 1.6178187131881714 | Accuracy: 19.53125%
ERM | Epoch 0 | Loss: 1.6105000972747803 | Accuracy: 21.09375%
ERM | Epoch 0 | Loss: 1.6174962520599365 | Accuracy: 14.84375%
ERM | Epoch 0 | Loss: 1.6157418489456177 | Accuracy: 17.96875%
ERM | Epoch 0 | Loss: 1.6115429401397705 | Accuracy: 15.625%
ERM | Epoch 0 | Loss: 1.6077340841293335 | Accuracy: 29.6875%
ERM | Epoch 0 | Loss: 1.6035034656524658 | Accuracy: 34.375%
ERM | Epoch 0 | Loss: 1.6152019500732422 | Accuracy: 32.03125%
ERM | Epoch 0 | Loss: 1.6062915325164795 | Accuracy: 19.53125%
ERM | Epoch 0 | Loss: 1.6034969091415405 | Accuracy: 22.65625%
ERM | Epoch 0 | Loss: 1.6070363521575928 | Accuracy: 17.96875%
ERM | Epoch 0 | Loss: 1.615973711013794 | Accuracy: 14.84375%
ERM | Epoch 0 | Loss: 1.6025010347366333 | Accuracy: 27.34375%
ERM | Epoch 0 | Loss: 1.598625659942627 | Accuracy: 21.09375%
ERM | Epoch 0 | Loss: 1.5956850051879883 | Accuracy: 25.0%
ERM | Epoch 0 | Loss: 1.6021933555603027 | Accuracy: 17.96875%
ERM

Epoch 0:  19%|█▉        | 71/376 [00:01<00:02, 104.50batch/s, accuracy=100.0%, loss=0.00539]

ERM | Epoch 0 | Loss: 1.2833702564239502 | Accuracy: 87.5%
ERM | Epoch 0 | Loss: 1.2387611865997314 | Accuracy: 84.375%
ERM | Epoch 0 | Loss: 1.1819055080413818 | Accuracy: 76.5625%
ERM | Epoch 0 | Loss: 1.065885305404663 | Accuracy: 81.25%
ERM | Epoch 0 | Loss: 0.9672184586524963 | Accuracy: 78.125%
ERM | Epoch 0 | Loss: 0.8504812717437744 | Accuracy: 78.90625%
ERM | Epoch 0 | Loss: 0.6989572048187256 | Accuracy: 84.375%
ERM | Epoch 0 | Loss: 0.6607310771942139 | Accuracy: 79.6875%
ERM | Epoch 0 | Loss: 0.4896869659423828 | Accuracy: 78.90625%
ERM | Epoch 0 | Loss: 0.39853060245513916 | Accuracy: 98.4375%
ERM | Epoch 0 | Loss: 0.2618454694747925 | Accuracy: 100.0%
ERM | Epoch 0 | Loss: 0.22547173500061035 | Accuracy: 99.21875%
ERM | Epoch 0 | Loss: 0.13949894905090332 | Accuracy: 100.0%
ERM | Epoch 0 | Loss: 0.07345389574766159 | Accuracy: 100.0%
ERM | Epoch 0 | Loss: 0.04286806285381317 | Accuracy: 100.0%
ERM | Epoch 0 | Loss: 0.026457488536834717 | Accuracy: 100.0%
ERM | Epoch 0 | L

Epoch 0:  32%|███▏      | 120/376 [00:01<00:01, 160.60batch/s, accuracy=100.0%, loss=0.0126]   

...

ERM | Epoch 19 | Loss: 0.011046762578189373 | Accuracy: 100.0%
ERM | Epoch 19 | Loss: 0.003952814731746912 | Accuracy: 100.0%
ERM | Epoch 19 | Loss: 0.0030735130421817303 | Accuracy: 100.0%
ERM | Epoch 19 | Loss: 0.04396038502454758 | Accuracy: 99.21875%
ERM | Epoch 19 | Loss: 0.0032888262066990137 | Accuracy: 100.0%
ERM | Epoch 19 | Loss: 0.035881441086530685 | Accuracy: 99.21875%
ERM | Epoch 19 | Loss: 0.09854477643966675 | Accuracy: 98.4375%
ERM | Epoch 19 | Loss: 0.003725644899532199 | Accuracy: 100.0%
ERM | Epoch 19 | Loss: 0.003561784978955984 | Accuracy: 100.0%
ERM | Epoch 19 | Loss: 0.003310291562229395 | Accuracy: 100.0%
ERM | Epoch 19 | Loss: 0.0684894397854805 | Accuracy: 99.21875%
ERM | Epoch 19 | Loss: 0.06252268701791763 | Accuracy: 99.21875%
ERM | Epoch 19 | Loss: 0.047388311475515366 | Accuracy: 99.21875%
ERM | Epoch 19 | Loss: 0.002562964567914605 | Accuracy: 100.0%
ERM | Epoch 19 | Loss: 0.02352493442595005 | Accuracy: 99.21875%
ERM | Epoch 19 | Loss: 0.00494634127244

Epoch 19:  91%|█████████ | 341/376 [00:01<00:00, 226.17batch/s, accuracy=99.21875%, loss=0.0403]

ERM | Epoch 19 | Loss: 0.013867007568478584 | Accuracy: 100.0%
ERM | Epoch 19 | Loss: 0.004457654897123575 | Accuracy: 100.0%
ERM | Epoch 19 | Loss: 0.09289134293794632 | Accuracy: 98.4375%
ERM | Epoch 19 | Loss: 0.002784819109365344 | Accuracy: 100.0%
ERM | Epoch 19 | Loss: 0.003787520807236433 | Accuracy: 100.0%
ERM | Epoch 19 | Loss: 0.0029571386985480785 | Accuracy: 100.0%
ERM | Epoch 19 | Loss: 0.14910605549812317 | Accuracy: 97.65625%
ERM | Epoch 19 | Loss: 0.0029904190450906754 | Accuracy: 100.0%
ERM | Epoch 19 | Loss: 0.0028308737091720104 | Accuracy: 100.0%
ERM | Epoch 19 | Loss: 0.052198950201272964 | Accuracy: 99.21875%
ERM | Epoch 19 | Loss: 0.1063222661614418 | Accuracy: 98.4375%
ERM | Epoch 19 | Loss: 0.004575151018798351 | Accuracy: 100.0%
ERM | Epoch 19 | Loss: 0.0057092029601335526 | Accuracy: 100.0%
ERM | Epoch 19 | Loss: 0.06709252297878265 | Accuracy: 98.4375%
ERM | Epoch 19 | Loss: 0.04616239294409752 | Accuracy: 99.21875%
ERM | Epoch 19 | Loss: 0.00586510123685002

Epoch 19: 100%|██████████| 376/376 [00:01<00:00, 190.66batch/s, accuracy=100.0%, loss=0.00196]

ERM | Epoch 19 | Loss: 0.005867974366992712 | Accuracy: 100.0%
ERM | Epoch 19 | Loss: 0.04470410197973251 | Accuracy: 99.21875%
ERM | Epoch 19 | Loss: 0.00577830895781517 | Accuracy: 100.0%
ERM | Epoch 19 | Loss: 0.005488098133355379 | Accuracy: 100.0%
ERM | Epoch 19 | Loss: 0.006623363122344017 | Accuracy: 100.0%
ERM | Epoch 19 | Loss: 0.05495956912636757 | Accuracy: 99.21875%
ERM | Epoch 19 | Loss: 0.06642208993434906 | Accuracy: 99.21875%
ERM | Epoch 19 | Loss: 0.005458206869661808 | Accuracy: 100.0%
ERM | Epoch 19 | Loss: 0.005167778581380844 | Accuracy: 100.0%
ERM | Epoch 19 | Loss: 0.0045554786920547485 | Accuracy: 100.0%
ERM | Epoch 19 | Loss: 0.004004074726253748 | Accuracy: 100.0%
ERM | Epoch 19 | Loss: 0.0033893343061208725 | Accuracy: 100.0%
ERM | Epoch 19 | Loss: 0.004155659582465887 | Accuracy: 100.0%
ERM | Epoch 19 | Loss: 0.0019577352795749903 | Accuracy: 100.0%


In [ ]:
Z = ermtrainer.trainer.get_trainset_outputs(features=True)
final_layer = ermtrainer.trainer.get_trainset_outputs(features=False)
print()
print(final_layer.shape)
print(Z.shape)
cpu_z = Z.cpu().numpy()


Getting model outputs: 100%|██████████| 751/751 [00:01<00:00, 475.11batch/s]


torch.Size([48004, 5])
torch.Size([48004, 84])


In [ ]:
def clustering_algorithm(z_val):
    normal_z = StandardScaler().fit_transform(z_val)
    umapped_z = umap.UMAP(n_components=8,n_neighbors=10).fit_transform(normal_z)
    print(normal_z.shape)
    print(umapped_z.shape)
    labels = convert_labels_to_partition(trainset.labels)
    final_partition = dict()
    for c in labels.keys():
        classZ = umapped_z[labels[c]]
        overclusterer = OverclusterModel(
            cluster_method = ClusterAlg.KMEANS.value,
            max_k = 5
        )
        overclusterer.fit(classZ)
        predictions = overclusterer.predict(classZ)
        cluster_to_idc = convert_labels_to_partition(predictions)
        global_cluster_partition = dict()
        for cluster_id, class_relative_indices in cluster_to_idc.items():
            global_indices = [labels[c][i] for i in class_relative_indices]
            global_cluster_partition[cluster_id] = global_indices
        for cluster_id, global_indices in global_cluster_partition.items():
            final_partition[(c, cluster_id)] = global_indices
    return final_partition

In [ ]:
final_cpartitions = clustering_algorithm(cpu_z)
totalsize = 0
for key, value in final_cpartitions.items():
  print(key,len(value))
  totalsize+=len(value)
print("totalsize: ",totalsize)

(48004, 84)
(48004, 8)
(2, np.int64(0)) 1180
(2, np.int64(5)) 3169
(2, np.int64(6)) 714
(2, np.int64(3)) 741
(2, np.int64(2)) 2698
(2, np.int64(1)) 480
(2, np.int64(4)) 29
(0, np.int64(0)) 4796
(0, np.int64(3)) 987
(0, np.int64(2)) 1559
(0, np.int64(1)) 2791
(4, np.int64(0)) 4923
(4, np.int64(1)) 3317
(4, np.int64(2)) 1201
(1, np.int64(1)) 5168
(1, np.int64(0)) 4504
(3, np.int64(3)) 2456
(3, np.int64(2)) 3603
(3, np.int64(0)) 2609
(3, np.int64(1)) 714
(3, np.int64(4)) 365
totalsize:  48004


In [ ]:
evaluator = Evaluator(
    testset=trainset,
    group_partition=final_cpartitions,
    group_weights=trainset.group_weights,
    batch_size=128,
    model=model,
    device=device,
    verbose=True
)
evaluator.evaluate()

Evaluating group-wise accuracy:   5%|▍         | 1/21 [00:00<00:07,  2.55it/s]

Group (0, np.int64(0)) Accuracy: 99.9582985821518


Evaluating group-wise accuracy:  10%|▉         | 2/21 [00:00<00:06,  2.76it/s]

Group (0, np.int64(1)) Accuracy: 98.67431028305266


Evaluating group-wise accuracy:  14%|█▍        | 3/21 [00:01<00:06,  2.90it/s]

...

Evaluating group-wise accuracy:  95%|█████████▌| 20/21 [00:06<00:00,  2.88it/s]

Group (4, np.int64(1)) Accuracy: 99.57793186614411


Evaluating group-wise accuracy: 100%|██████████| 21/21 [00:07<00:00,  2.96it/s]

Group (4, np.int64(2)) Accuracy: 100.0


{(0, np.int64(0)): 99.9582985821518,
 (0, np.int64(1)): 98.67431028305266,
 (0, np.int64(2)): 100.0,
 (0, np.int64(3)): 98.78419452887537,
 (1, np.int64(0)): 98.97868561278864,
 (1, np.int64(1)): 100.0,
 (2, np.int64(0)): 98.89830508474576,
 (2, np.int64(1)): 100.0,
 (2, np.int64(2)): 99.85174203113418,
 (2, np.int64(3)): 100.0,
 (2, np.int64(4)): 0.0,
 (2, np.int64(5)): 100.0,
 (2, np.int64(6)): 100.0,
 (3, np.int64(0)): 99.57838252203909,
 (3, np.int64(1)): 100.0,
 (3, np.int64(2)): 98.94532334165973,
 (3, np.int64(3)): 100.0,
 (3, np.int64(4)): 100.0,
 (4, np.int64(0)): 99.32967702620354,
 (4, np.int64(1)): 99.57793186614411,
 (4, np.int64(2)): 100.0}

In [ ]:
george_model = model_factory("lenet", trainset[0][0].shape, trainset.num_classes).to(device)
robust_trainset = GroupLabeledDatasetWrapper(trainset, final_cpartitions)
gdro_trainer = GroupDRO(
    model=george_model,
    num_epochs=20,
    trainset=robust_trainset,
    batch_size=128,
    optimizer=SGD(george_model.parameters(), lr=0.01, momentum=0.9, nesterov=True),
    device=device,
    verbose=True
)
gdro_trainer.train()


Epoch 0:   2%|▏         | 7/376 [00:00<00:25, 14.58batch/s, accuracy=30.46875%, loss=1.57]

 | Epoch 0 | Loss: 1.6030086278915405 | Accuracy: 20.3125%
 | Epoch 0 | Loss: 1.6007801294326782 | Accuracy: 14.0625%
 | Epoch 0 | Loss: 1.5989727973937988 | Accuracy: 11.71875%
 | Epoch 0 | Loss: 1.5965967178344727 | Accuracy: 19.53125%
 | Epoch 0 | Loss: 1.5202398300170898 | Accuracy: 39.0625%
 | Epoch 0 | Loss: 1.5181267261505127 | Accuracy: 49.21875%
 | Epoch 0 | Loss: 1.5882017612457275 | Accuracy: 36.71875%
 | Epoch 0 | Loss: 1.5839331150054932 | Accuracy: 32.8125%
 | Epoch 0 | Loss: 1.5798207521438599 | Accuracy: 32.03125%
 | Epoch 0 | Loss: 1.5764881372451782 | Accuracy: 34.375%
 | Epoch 0 | Loss: 1.5735199451446533 | Accuracy: 25.0%
 | Epoch 0 | Loss: 1.56856107711792 | Accuracy: 30.46875%


Epoch 0:   5%|▌         | 19/376 [00:00<00:10, 32.78batch/s, accuracy=32.03125%, loss=1.51]

 | Epoch 0 | Loss: 1.5640912055969238 | Accuracy: 28.90625%
 | Epoch 0 | Loss: 1.4872207641601562 | Accuracy: 27.34375%
 | Epoch 0 | Loss: 1.5565656423568726 | Accuracy: 36.71875%
 | Epoch 0 | Loss: 1.5519970655441284 | Accuracy: 27.34375%
 | Epoch 0 | Loss: 1.547628402709961 | Accuracy: 35.9375%
 | Epoch 0 | Loss: 1.54296875 | Accuracy: 27.34375%
 | Epoch 0 | Loss: 1.5401649475097656 | Accuracy: 37.5%
 | Epoch 0 | Loss: 1.5343880653381348 | Accuracy: 35.15625%
 | Epoch 0 | Loss: 1.5292134284973145 | Accuracy: 32.03125%
 | Epoch 0 | Loss: 1.522713541984558 | Accuracy: 32.8125%
 | Epoch 0 | Loss: 1.5207611322402954 | Accuracy: 29.6875%
 | Epoch 0 | Loss: 1.5126794576644897 | Accuracy: 32.03125%


Epoch 0:   8%|▊         | 31/376 [00:01<00:07, 43.33batch/s, accuracy=31.25%, loss=1.39]   

...

 | Epoch 19 | Loss: 0.0027388499584048986 | Accuracy: 100.0%
 | Epoch 19 | Loss: 0.0062235076911747456 | Accuracy: 100.0%
 | Epoch 19 | Loss: 0.024981115013360977 | Accuracy: 99.21875%
 | Epoch 19 | Loss: 0.016775144264101982 | Accuracy: 99.21875%
 | Epoch 19 | Loss: 0.03507330268621445 | Accuracy: 99.21875%
 | Epoch 19 | Loss: 0.010951037518680096 | Accuracy: 100.0%
 | Epoch 19 | Loss: 0.050643160939216614 | Accuracy: 99.21875%
 | Epoch 19 | Loss: 0.07206588238477707 | Accuracy: 99.21875%
 | Epoch 19 | Loss: 0.003959007561206818 | Accuracy: 100.0%
 | Epoch 19 | Loss: 0.00609554722905159 | Accuracy: 100.0%
 | Epoch 19 | Loss: 0.00787042360752821 | Accuracy: 100.0%


Epoch 19:  98%|█████████▊| 367/376 [00:07<00:00, 54.10batch/s, accuracy=100.0%, loss=0.00142]

 | Epoch 19 | Loss: 0.019301127642393112 | Accuracy: 100.0%
 | Epoch 19 | Loss: 0.002636105753481388 | Accuracy: 100.0%
 | Epoch 19 | Loss: 0.019082192331552505 | Accuracy: 99.21875%
 | Epoch 19 | Loss: 0.017342031002044678 | Accuracy: 100.0%
 | Epoch 19 | Loss: 0.0024316231720149517 | Accuracy: 100.0%
 | Epoch 19 | Loss: 0.005102692171931267 | Accuracy: 100.0%
 | Epoch 19 | Loss: 0.0021984560880810022 | Accuracy: 100.0%
 | Epoch 19 | Loss: 0.034269049763679504 | Accuracy: 99.21875%
 | Epoch 19 | Loss: 0.08039958029985428 | Accuracy: 99.21875%
 | Epoch 19 | Loss: 0.011838344857096672 | Accuracy: 100.0%
 | Epoch 19 | Loss: 0.0015732593601569533 | Accuracy: 100.0%
 | Epoch 19 | Loss: 0.0014214899856597185 | Accuracy: 100.0%


Epoch 19: 100%|██████████| 376/376 [00:07<00:00, 51.32batch/s, accuracy=100.0%, loss=0.0096]

 | Epoch 19 | Loss: 0.0047057135961949825 | Accuracy: 100.0%
 | Epoch 19 | Loss: 0.029832279309630394 | Accuracy: 99.21875%
 | Epoch 19 | Loss: 0.013570640236139297 | Accuracy: 99.21875%
 | Epoch 19 | Loss: 0.0010540725197643042 | Accuracy: 100.0%
 | Epoch 19 | Loss: 0.11708138883113861 | Accuracy: 99.21875%
 | Epoch 19 | Loss: 0.009597457014024258 | Accuracy: 100.0%


Baseline model test accuracies on groups

In [ ]:
evaluator = Evaluator(
    testset=testset,
    group_partition=testset.group_partition,
    group_weights=testset.group_weights,
    batch_size=64,
    model=model,
    device=device,
    verbose=True
)
evaluator.evaluate()

Evaluating group-wise accuracy:   4%|▍         | 1/25 [00:00<00:07,  3.24it/s]

Group (0, 0) Accuracy: 100.0


Evaluating group-wise accuracy:   8%|▊         | 2/25 [00:00<00:07,  3.26it/s]

Group (0, 1) Accuracy: 0.0


Evaluating group-wise accuracy:  12%|█▏        | 3/25 [00:00<00:06,  3.17it/s]

...

Evaluating group-wise accuracy:  96%|█████████▌| 24/25 [00:07<00:00,  3.13it/s]

Group (4, 3) Accuracy: 0.0


Evaluating group-wise accuracy: 100%|██████████| 25/25 [00:07<00:00,  3.17it/s]

Group (4, 4) Accuracy: 100.0


{(0, 0): 100.0,
 (0, 1): 0.0,
 (0, 2): 0.0,
 (0, 3): 0.0,
 (0, 4): 17.257683215130022,
 (1, 0): 2.2004889975550124,
 (1, 1): 100.0,
 (1, 2): 0.0,
 (1, 3): 0.0,
 (1, 4): 0.0,
 (2, 0): 0.0,
 (2, 1): 0.26666666666666666,
 (2, 2): 100.0,
 (2, 3): 0.0,
 (2, 4): 0.0,
 (3, 0): 0.0,
 (3, 1): 0.0,
 (3, 2): 0.0,
 (3, 3): 100.0,
 (3, 4): 0.2518891687657431,
 (4, 0): 17.632241813602015,
 (4, 1): 0.0,
 (4, 2): 0.0,
 (4, 3): 0.0,
 (4, 4): 100.0}

Evaluator Trained on GDRO with George Test Accuracies

In [ ]:
evaluator = Evaluator(
    testset=testset,
    group_partition=testset.group_partition,
    group_weights=testset.group_weights,
    batch_size=64,
    model=gdro_trainer.trainer.model,
    device=device,
    verbose=True
)
evaluator.evaluate()

Evaluating group-wise accuracy:   4%|▍         | 1/25 [00:00<00:07,  3.17it/s]

Group (0, 0) Accuracy: 100.0


Evaluating group-wise accuracy:   8%|▊         | 2/25 [00:00<00:07,  3.16it/s]

Group (0, 1) Accuracy: 69.97635933806147


Evaluating group-wise accuracy:  12%|█▏        | 3/25 [00:00<00:06,  3.21it/s]

...

Evaluating group-wise accuracy:  96%|█████████▌| 24/25 [00:07<00:00,  3.18it/s]

Group (4, 3) Accuracy: 4.797979797979798


Evaluating group-wise accuracy: 100%|██████████| 25/25 [00:07<00:00,  3.17it/s]

Group (4, 4) Accuracy: 100.0


{(0, 0): 100.0,
 (0, 1): 69.97635933806147,
 (0, 2): 27.18676122931442,
 (0, 3): 9.21985815602837,
 (0, 4): 63.12056737588652,
 (1, 0): 55.74572127139364,
 (1, 1): 100.0,
 (1, 2): 48.03921568627451,
 (1, 3): 20.833333333333332,
 (1, 4): 0.0,
 (2, 0): 56.0,
 (2, 1): 60.0,
 (2, 2): 100.0,
 (2, 3): 23.466666666666665,
 (2, 4): 24.06417112299465,
 (3, 0): 24.87437185929648,
 (3, 1): 4.534005037783375,
 (3, 2): 31.738035264483628,
 (3, 3): 99.74811083123426,
 (3, 4): 19.64735516372796,
 (4, 0): 51.13350125944584,
 (4, 1): 4.785894206549118,
 (4, 2): 1.2594458438287153,
 (4, 3): 4.797979797979798,
 (4, 4): 100.0}